# MongoDB Field Level Encryption Demo

In [ ]:
import pymongo
from pymongo import MongoClient
from pymongo.encryption import ClientEncryption
from bson import Binary
import pymongocrypt
import base64
from bson.codec_options import DEFAULT_CODEC_OPTIONS

### General Settings:

Generate CMK Example Shell Command: head -c 96 /dev/urandom | base64 > key.txt

In [ ]:
cmk = base64.b64decode("IkhJDuh1kQXt7cNlrBkKEMpV/jbpUERloqPGCB0WaNhFC7Ig4z1X2dDwv6XByMlrRyadd3MEC+BdcY8D9cadCgvHe3MVa8o1m00+xcE4KDFJaID/FS1rwZKRPK3KfezP")

In [ ]:
kms_providers = { "local" : { "key" : cmk } }
database = 'hr'
keyStore = 'dataKeys'
key_vault_namespace = database + '.' + keyStore

### Create Data Encryption Keys:

In [ ]:
keyVaultClient = pymongo.MongoClient("127.0.0.1")

In [ ]:
keyVault = ClientEncryption(kms_providers, key_vault_namespace, keyVaultClient, DEFAULT_CODEC_OPTIONS)

Create two data encryption keys in the keys collection:

In [ ]:
keyVault.create_data_key("local", master_key=cmk, key_alt_names=["key1"])

In [ ]:
keyVault.create_data_key("local", master_key=cmk, key_alt_names=["key2"])

### Create Encryption Schema:

Retrieve the key _id to be referenced in the schema_map:

In [ ]:
keyVaultClient = pymongo.MongoClient("127.0.0.1")

In [ ]:
keyDb = keyVaultClient.hr
key1 = keyDb[keyStore].find_one({'keyAltNames':'key1'})['_id']
key2 = keyDb[keyStore].find_one({'keyAltNames':'key2'})['_id']

Define the encryption schema:

In [ ]:
schema_map = {
    "hr.employees":
    {
        'bsonType': 'object',
        'encryptMetadata': {
            'keyId': [key1],
            'algorithm': 'AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic'
            },
        'properties': {
            'ssn': {
                'encrypt': {
                    'bsonType': 'string'
                    }
                },
            'position': {
                'bsonType': 'object',
                'encryptMetadata': {
                    'keyId': [key2],
                    'algorithm': 'AEAD_AES_256_CBC_HMAC_SHA_512-Random'
                    },
                'properties': {
                    'compensation': {
                        'encrypt': {}
                        }
                }}
            }
        }
    }

Configure the field level auto encryption client:

In [ ]:
fleOptions = pymongo.encryption_options.AutoEncryptionOpts(kms_providers, key_vault_namespace, key_vault_client=None, schema_map=schema_map, bypass_auto_encryption=False, mongocryptd_uri='mongodb://localhost:27020', mongocryptd_bypass_spawn=False, mongocryptd_spawn_path='mongocryptd', mongocryptd_spawn_args=None)

Initialise the secured client:

In [ ]:
autoEncClient = pymongo.MongoClient("127.0.0.1",auto_encryption_opts = fleOptions)

### Insert Data

As defined in the schema_map the field 'ssn' and 'compensation' will be encrypted.

In [ ]:
employee = {
    'name': 'John Doe',
    'address': {
        'street': '1234 Main Street',
        'city': 'MongoDBVille',
        'zip': 99999
    },
    'phone': '949-555-1212',
    'ssn': '123-45-6789',
    'position': {'compensation': 1234}
}

In [ ]:
db = autoEncClient.hr
db.employees.insert_one(employee)

### Read Data

Deterministic encryption allows to find encrypted data like you are used to with equality matches:

In [ ]:
db = autoEncClient.hr
db.employees.find_one({'ssn': '123-45-6789'})

### Use Compass or the following to show the encrypted values

Clients without access to the keystore can only retrieve encrypted data values and cannot search for encrypted fields.

In [ ]:
noEncClient = pymongo.MongoClient("127.0.0.1")
db = noEncClient.hr
db.employees.find_one({'ssn': '123-45-6789'})

In [ ]:
db.employees.find_one({'name': 'John Doe'})

### Cleanup Environment

In [ ]:
cleanClient = pymongo.MongoClient("127.0.0.1")
cleanClient.drop_database(database)